In [1]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'

In [2]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
split = 2

In [4]:
adata_preds = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/adata_with_predictions_{split}.h5ad")

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [5]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [6]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [7]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [8]:
adata_pred_test = adata_preds[adata_preds.obs["split"]=="test"]
adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["CPA_pred"]
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

adata_pred_ood = adata_preds[adata_preds.obs["split"]=="ood"]
adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["CPA_pred"]
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

/tmp/ipykernel_1717315/2350350615.py:2: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
/tmp/ipykernel_1717315/2350350615.py:18: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]


In [9]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [10]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [11]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [12]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [13]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [14]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.6575102380343846,
 'decoded_ood_sinkhorn_div_1': 112.96990530831474,
 'decoded_ood_sinkhorn_div_10': 79.96565355573382,
 'decoded_ood_sinkhorn_div_100': 33.36105564662388,
 'decoded_ood_e_distance': 65.87899291656133,
 'decoded_ood_mmd': 0.22227964656693594}

In [15]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7855887512365977,
 'decoded_test_e_distance': 42.35082873994283,
 'decoded_test_mmd_distance': 0.205386554201444}

In [16]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.6575102380343846,
 'decoded_ood_sinkhorn_div_1': 112.96990530831474,
 'decoded_ood_sinkhorn_div_10': 79.96565355573382,
 'decoded_ood_sinkhorn_div_100': 33.36105564662388,
 'decoded_ood_e_distance': 65.87899291656133,
 'decoded_ood_mmd': 0.22227964656693594}

In [17]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': -2.4244329163006375,
 'encoded_ood_sinkhorn_div_1': 35.92998729433332,
 'encoded_ood_sinkhorn_div_10': 32.66790199279785,
 'encoded_ood_sinkhorn_div_100': 31.524854387555802,
 'encoded_ood_e_distance': 62.78804557977087,
 'encoded_ood_mmd': 0.25390260560171946}

In [18]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.4122078674180167,
 'deg_ood_sinkhorn_div_1': 28.785127299172537,
 'deg_ood_sinkhorn_div_10': 20.15852669307164,
 'deg_ood_sinkhorn_div_100': 19.513290677751815,
 'deg_ood_e_distance': 38.894598632316615,
 'deg_ood_mmd': 0.3057304450443813}

In [19]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.623411682744821,
 'deg_test_sinkhorn_div_1': 21.077818900346756,
 'deg_test_sinkhorn_div_10': 12.935458779335022,
 'deg_test_sinkhorn_div_100': 11.944055457909903,
 'deg_test_e_distance': 23.74693614360884,
 'deg_test_mmd': 0.2751287283996741}

In [20]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7855887512365977,
 'decoded_test_e_distance': 42.35082873994283,
 'decoded_test_mmd_distance': 0.205386554201444}

In [21]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.6575102380343846,
 'decoded_ood_sinkhorn_div_1': 112.96990530831474,
 'decoded_ood_sinkhorn_div_10': 79.96565355573382,
 'decoded_ood_sinkhorn_div_100': 33.36105564662388,
 'decoded_ood_e_distance': 65.87899291656133,
 'decoded_ood_mmd': 0.22227964656693594}

In [22]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.4122078674180167,
 'deg_ood_sinkhorn_div_1': 28.785127299172537,
 'deg_ood_sinkhorn_div_10': 20.15852669307164,
 'deg_ood_sinkhorn_div_100': 19.513290677751815,
 'deg_ood_e_distance': 38.894598632316615,
 'deg_ood_mmd': 0.3057304450443813}

In [23]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7855887512365977,
 'decoded_test_e_distance': 42.35082873994283,
 'decoded_test_mmd_distance': 0.205386554201444}

In [24]:
mean_test_metrics_encoded

{'encoded_test_r_squared': -0.1203614710204771,
 'encoded_test_sinkhorn_div_1': 24.285676548878353,
 'encoded_test_sinkhorn_div_10': 20.776190916697185,
 'encoded_test_sinkhorn_div_100': 19.636349320411682,
 'encoded_test_e_distance': 39.02329327154936,
 'encoded_test_mmd': 0.18478726704294482}

In [25]:
ood_metrics_encoded

{'Dacinostat+Dasatinib': {'r_squared': -5.44976806640625,
  'sinkhorn_div_1': 31.059206008911133,
  'sinkhorn_div_10': 27.891632080078125,
  'sinkhorn_div_100': 27.046600341796875,
  'e_distance': 53.92180112768126,
  'mmd': 0.27600548},
 'Dacinostat+PCI-34051': {'r_squared': -5.304579734802246,
  'sinkhorn_div_1': 32.21506881713867,
  'sinkhorn_div_10': 29.104515075683594,
  'sinkhorn_div_100': 28.228431701660156,
  'e_distance': 56.26879687267267,
  'mmd': 0.27837935},
 'Givinostat+Cediranib': {'r_squared': 0.8949863910675049,
  'sinkhorn_div_1': 3.752619743347168,
  'sinkhorn_div_10': 2.0727548599243164,
  'sinkhorn_div_100': 1.5300168991088867,
  'e_distance': 2.9526262636819443,
  'mmd': 0.07301652},
 'Givinostat+Curcumin': {'r_squared': 0.9327555298805237,
  'sinkhorn_div_1': 3.224982738494873,
  'sinkhorn_div_10': 1.5919313430786133,
  'sinkhorn_div_100': 1.0309152603149414,
  'e_distance': 1.9312919302201568,
  'mmd': 0.067073144},
 'Panobinostat+Alvespimycin': {'r_squared': -1

In [26]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/cpa"

In [27]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
